<a href="https://colab.research.google.com/github/Ninyago53/chess-game-generator/blob/main/chess_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y wget
!wget https://stockfishchess.org/files/stockfish_14_linux_x64.zip
!unzip stockfish_14_linux_x64.zip

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Hit:14 http://ppa.launchpad.net/u

In [2]:
!pip install chess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 KB 1.4 MB/s eta 0:00:00


In [3]:
import chess
import chess.engine
import pandas as pd

# Function to generate a chess game
def generate_game(time_per_move):
    engine = chess.engine.SimpleEngine.popen_uci("./stockfish_14_linux_x64/stockfish_14_x64")
    board = chess.Board()
    
    moves = []

    while not board.is_game_over():

        result = engine.play(board, chess.engine.Limit(time=time_per_move))

        move = result.move
        uci_move = move.uci() # Get the move in UCI format
        moves.append(uci_move)

        board.push(move)

    engine.quit()
    # print(moves)
    return moves

# Function to generate multiple games and store the results in a DataFrame
def generate_games(num_games, time_per_move):
    df = pd.DataFrame(columns=["Game", "Moves"])

    for i in range(num_games):
        moves = generate_game(time_per_move)
        df = pd.concat([df, pd.DataFrame({"Game": [i], "Moves": [moves]})], ignore_index=True)
    df.to_csv("chess_games.csv", index=False)
    return df

# Example usage: generate 1 games with 0.5 second per move
df = generate_games(1, 0.5)
print(df)


  Game                                              Moves
0    0  [d2d4, d7d5, c2c4, e7e6, g1f3, d5c4, e2e3, g8f...


In [ ]:
import chess
import chess.engine
import pandas as pd

# Create Chess engine
engine = chess.engine.SimpleEngine.popen_uci("./stockfish_14_linux_x64/stockfish_14_x64")

# Create Chess board
board = chess.Board()

# Initialize empty DataFrame for positions and moves
positions_and_moves_df = pd.DataFrame(columns=["position", "move"])

# Add current board position as first row to the DataFrame
positions_and_moves_df = positions_and_moves_df.append({"position": board.fen(), "move": ""}, ignore_index=True)

# Play Chess game
while not board.is_game_over():
    # Let Stockfish compute best move
    result = engine.play(board, chess.engine.Limit(time=2.0))
    move = result.move

    # Add current board position and suggested move to the DataFrame
    positions_and_moves_df = positions_and_moves_df.append({"position": board.fen(), "move": move.uci()}, ignore_index=True)

    # Execute move on the Chess board
    board.push(move)

# Save DataFrame as a Parquet file
positions_and_moves_df.to_parquet("positions_and_moves.parquet")

# Close Chess engine
engine.quit()


In [ ]:
import pyarrow.parquet as pq
import pandas as pd

file_path = '/content/positions_and_moves.parquet'
df = pd.read_parquet(file_path)

print(df)


In [ ]:
import csv

file_path = '/content/chess_games.csv'

with open(file_path, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in csvreader:
        print(', '.join(row))
